In [15]:
import requests
import pandas as pd
import pytz
import config

In [16]:
api_key = config.api_key 
print(api_key)
bbox = '-6.15450317,53.30408682,-6.07896861,53.34080955'
satellites = 'Sentinel-1A, Sentinel-1B, Sentinel-2A, Sentinel-2B, Sentinel-3A, Sentinel-3B,Landsat-8'

url = 'https://api.spectator.earth/overpass/?api_key={api_key}&bbox={bbox}&satellites={satellites}'.format(
    api_key=api_key, bbox=bbox, satellites=satellites)

response = requests.get(url)
data = response.json()
print(data)

2beb4e7e1c9379fbbda4301bca0f31645799f353972f4ecdf2fc57a31dd42762
{'errors': ['Authentication credentials were not provided.']}


In [ ]:
result = []
for i in range(len(data['overpasses'])):
  overpass = {}
  overpass['date'] = data['overpasses'][i]['date']
  overpass['satellite'] = data['overpasses'][i]['satellite']
  print(overpass)
  result.append(overpass)

In [6]:
data = pd.DataFrame(result, columns=['satellite', 'date'])
data['Day'] = pd.to_datetime(data['date']).dt.day_name()
data['Date'] = pd.to_datetime(data['date']).dt.date
data['IST'] = pd.DatetimeIndex(pd.to_datetime(data['date'])).tz_convert('Europe/Dublin')
data['UTC'] = pd.to_datetime(data['date']).dt.time
data['IST'] = pd.to_datetime(data['IST']).dt.time
data.pop('date')
data = data.rename(columns={'satellite': 'Satellite'})

In [7]:
data['Date'] = data['Date'].astype(str)
data['IST'] = data['IST'].astype(str)
data['UTC'] = data['UTC'].astype(str)

In [8]:
data.head()

,Satellite,Day,Date,IST,UTC
0,Sentinel-1B,Tuesday,2021-07-06,19:14:00,18:14:00
1,Sentinel-3A,Tuesday,2021-07-06,22:25:00,21:25:00
2,Sentinel-2B,Tuesday,2021-07-06,23:11:00,22:11:00
3,Sentinel-1A,Wednesday,2021-07-07,07:56:00,06:56:00
4,Sentinel-3B,Wednesday,2021-07-07,11:33:00,10:33:00


In [10]:
scopes = ['https://spreadsheets.google.com/feeds',
          'https://www.googleapis.com/auth/drive',]

credentials = ServiceAccountCredentials.from_json_keyfile_name('/content/drive/MyDrive/Colab Notebooks/satelliteoverpasskey.json',scopes=scopes)
sheetid = '1hx5xmfN6G9hyoHuBxPAVrF75SU90OJ8ko_GVUAaGYds'

In [11]:
service = discovery.build('sheets', 'v4', credentials=credentials)
range_name="Sheet1!A1:D1"
body = {
    'majorDimension' : 'ROWS',
    'values': data.values.tolist()
}

In [ ]:
service.spreadsheets().values().append(
    spreadsheetId=sheetid, range=range_name, valueInputOption='USER_ENTERED',
    insertDataOption='INSERT_ROWS', body=body).execute()